In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import solve_ivp
from tqdm import tqdm
from scipy.optimize import minimize
import scipy as sp
from scipy.integrate import solve_ivp
import numdifftools as nd

from numpy.linalg import eig

In [ ]:
import os
if 'inference-from-testing' in os.getcwd():
    os.chdir("../..")
os.getcwd()

In [ ]:
from copy import deepcopy
from numpy import arange, array, atleast_2d, log
from os import mkdir
from os.path import isdir, isfile
from pickle import load, dump
from pandas import read_csv
from scipy.integrate import solve_ivp
from model.preprocessing import ( estimate_beta_ext, estimate_growth_rate,
        SEIRInput, HouseholdPopulation, make_initial_condition_by_eigenvector)
from model.specs import SINGLE_AGE_SEIR_SPEC_FOR_FITTING, SINGLE_AGE_UK_SPEC
from model.common import SEIRRateEquations
from model.imports import FixedImportModel

In [ ]:
DOUBLING_TIME = 3
growth_rate = log(2) / DOUBLING_TIME

comp_dist = read_csv(
    'inputs/england_hh_size_dist.csv',
    header=0).to_numpy().squeeze()
comp_dist = comp_dist[:3]
comp_dist[:2] *= 0
comp_dist = comp_dist/sum(comp_dist)
max_hh_size = len(comp_dist)
composition_list = np.atleast_2d(arange(1, max_hh_size+1)).T

#comp_dist = array([1.])
#composition_list = array([[1]])

if isdir('outputs') is False:
    mkdir('outputs')
if isdir('outputs/inference-from-testing') is False:
    mkdir('outputs/inference-from-testing')

SPEC = {**SINGLE_AGE_SEIR_SPEC_FOR_FITTING, **SINGLE_AGE_UK_SPEC}
model_input_to_fit = SEIRInput(SPEC, composition_list, comp_dist)
household_population_to_fit = HouseholdPopulation(
    composition_list, comp_dist, model_input_to_fit)
beta_ext = 0.
model_input = deepcopy(model_input_to_fit)
model_input.k_ext *= beta_ext

true_density_expo = model_input.density_expo


# With the parameters chosen, we calculate Q_int:
household_population = HouseholdPopulation(
    composition_list, comp_dist, model_input)

true_lam = 3.
pop_prev = 1e-2
rhs = SEIRRateEquations(model_input, household_population, FixedImportModel(4,1, np.array([true_lam * pop_prev])))

#r_est = estimate_growth_rate(household_population, rhs, [0.001, 5], 1e-9)


#print('Estimated growth rate is',r_est,'.')
#print('Estimated doubling time is',log(2) / r_est,'.')

H0 = np.zeros((household_population.total_size),)
all_sus = np.where(np.sum(rhs.states_exp_only + rhs.states_inf_only + rhs.states_rec_only, 1)<1e-1)[0]
one_inf = np.where((np.abs(np.sum(rhs.states_inf_only, 1) - 1)<1e-1) & (np.sum(rhs.states_exp_only + rhs.states_rec_only, 1)<1e-1))[0]
H0[all_sus] = 0.99 * comp_dist
H0[one_inf] = 0.01 * comp_dist
S0 = H0.T.dot(household_population.states[:, ::4])
E0 = H0.T.dot(household_population.states[:, 1::4])
I0 = H0.T.dot(household_population.states[:, 2::4])
R0 = H0.T.dot(household_population.states[:, 3::4])
start_state = (1/model_input.ave_hh_size) * array([S0.sum(),
                                                   E0.sum(),
                                                   I0.sum(),
                                                   R0.sum()])
tspan = (0.0, 365)
print(start_state)

In [ ]:
comp_dist

In [ ]:
composition_list

In [ ]:
# Now set up evaluation time points and solve system:

# New time at which we evaluate the infection
trange = np.arange(0,7*12,7) # Evaluate for 12 weeks

# Solve:
solution = solve_ivp(rhs, tspan, H0, first_step=0.001, atol=1e-16, t_eval=trange)

time = solution.t
H = solution.y
S = H.T.dot(household_population.states[:, ::4])
E = H.T.dot(household_population.states[:, 1::4])
I = H.T.dot(household_population.states[:, 2::4])
R = H.T.dot(household_population.states[:, 3::4])

In [ ]:
from matplotlib.pyplot import subplots
from matplotlib.cm import get_cmap

data_list = [S/model_input.ave_hh_by_class,
    E/model_input.ave_hh_by_class,
    I/model_input.ave_hh_by_class,
    R/model_input.ave_hh_by_class]

lgd=['S','E','I','R']

fig, (axis) = subplots(1,1, sharex=True)

cmap = get_cmap('tab20')
alpha = 0.5
for i in range(len(data_list)):
    axis.plot(
        time, data_list[i], label=lgd[i],
        color=cmap(i/len(data_list)), alpha=alpha)
axis.set_ylabel('Proportion of population')
axis.legend(ncol=1, bbox_to_anchor=(1,0.50))

In [ ]:
H[H<0.0] = 0.0

Make test data for single-household trajectories and calculate LLH

In [ ]:
from numpy.random import choice

H0 = make_initial_condition_by_eigenvector(growth_rate, model_input, household_population, rhs, 1e-1, 0.0,False,3)
test_times = np.arange(7,7*12,7)
def generate_single_hh_test_data(test_times):
    Ht = deepcopy(H0)
    test_data = np.zeros((len(test_times),))
    for i in range(len(test_times)-1):
        tspan = (test_times[i], test_times[i+1])
        solution = solve_ivp(rhs, tspan, Ht, first_step=0.001, atol=1e-16)
        time = solution.t
        H = solution.y
        state = choice(range(len(H[:, -1])), 1, p=H[:, -1]/sum(H[:, -1]))
        test_data[i] = rhs.states_inf_only[state]
        Ht *= 0
        Ht[state] = 1
    return(test_data)
sample_data = generate_single_hh_test_data(test_times)
print(sample_data)

In [ ]:
# Log likelihood if we can only measure infecteds
# This is for test results from one household, you´ll need to adapt to multiple

def llh_from_test_data(test_data, test_times, rhs, H0):
    Ht = deepcopy(H0)
    llh = 0
    for i in range(len(test_times)-1):
        if i==0:
            start_time = 0
        else:
            start_time = test_times[i-1]
        tspan = (start_time, test_times[i])
        solution = solve_ivp(rhs, tspan, Ht, first_step=0.001, atol=1e-16)
        time = solution.t
        H = solution.y
        I = test_data[i]
        possible_states = np.where(np.abs(np.sum(rhs.states_inf_only,1)-I)<1e-1)[0]
        llh += np.log(np.sum(H[possible_states, -1]))
        Ht *= 0
        Ht[possible_states] = H[possible_states, -1]
    return(llh)

llh_from_test_data(sample_data, test_times, rhs, H0)

Quick check to make sure the restricted trajectory locks on to the right thing:

In [ ]:
# Log likelihood calculation but return probability trajectory

def llh_with_traj(test_data, test_times, rhs, H0):
    Ht = deepcopy(H0)
    H_all = np.atleast_2d(deepcopy(H0)).T
    t_all = np.array(0)
    llh = 0
    for i in range(len(test_times)-1):
        if i==0:
            start_time = 0
        else:
            start_time = test_times[i-1]
        tspan = (start_time, test_times[i])
        solution = solve_ivp(rhs, tspan, Ht, first_step=0.001, atol=1e-16)
        time = solution.t
        H = solution.y
        I = test_data[i]
        possible_states = np.where(np.abs(np.sum(rhs.states_inf_only,1)-I)<1e-1)[0]
        llh += np.log(np.sum(H[possible_states, -1]))
        Ht *= 0
        Ht[possible_states] = H[possible_states, -1]
        Ht = Ht/sum(Ht)
        H_all = np.hstack((H_all, H))
        t_all = np.hstack((t_all, time))
    return(llh, H_all, t_all)

llh, H_all, t_all = llh_with_traj(sample_data, test_times, rhs, H0)

In [ ]:
S = H_all.T.dot(household_population.states[:, ::4])
E = H_all.T.dot(household_population.states[:, 1::4])
I = H_all.T.dot(household_population.states[:, 2::4])
R = H_all.T.dot(household_population.states[:, 3::4])

data_list = [S/model_input.ave_hh_by_class,
    E/model_input.ave_hh_by_class,
    I/model_input.ave_hh_by_class,
    R/model_input.ave_hh_by_class]

lgd=['S','E','I','R', "Test data"]

fig, (axis) = subplots(1,1, sharex=True)

cmap = get_cmap('tab20')
alpha = 0.5
for i in range(len(data_list)):
    axis.plot(t_all,
        3*data_list[i], label=lgd[i],
        color=cmap(i/len(data_list)), alpha=alpha)
axis.plot(test_times, sample_data, marker=".", ls="", ms=20, label=lgd[-1])
axis.set_ylabel('Proportion of population')
axis.legend(ncol=1, bbox_to_anchor=(1,0.50))

In [ ]:
# Now do multiple households

# Generate test data:
n_hh = 500
multi_hh_data = [generate_single_hh_test_data(test_times) for i in range(n_hh)]

# Option: Only include houses with at least one +ve
# multi_hh_data = [data for data in multi_hh_data if np.sum(data)>0]

In [ ]:
multi_hh_data

In [ ]:
# Try a single likelihood calculation

multi_hh_llh = sum(array([llh_from_test_data(sample_data, test_times, rhs, H0) for sample_data in multi_hh_data]))

At this point, we should be able to write a function taking parameters as input and calculating llh for those parameters.

In [ ]:
def llh_from_pars(data, test_times, tau, lam):
    model_input = SEIRInput(SPEC, composition_list, comp_dist, print_ests=False)
    model_input.k_home = (tau / model_input.beta_int) * model_input.k_home
    model_input.k_ext = 0 * model_input.k_ext
    model_input.density_expo = true_density_expo
    # With the parameters chosen, we calculate Q_int:
    household_population = HouseholdPopulation(
        composition_list, comp_dist, model_input)
    # lam goes in as arg for rate equations
    rhs = SEIRRateEquations(model_input, household_population, FixedImportModel(4,1, np.array([lam * pop_prev])))
    H0 = make_initial_condition_by_eigenvector(growth_rate, model_input, household_population, rhs, 1e-2, 0.0,False,3)
    tspan = (0.0, 365)
    return(sum(array([llh_from_test_data(data, test_times, rhs, H0) for data in multi_hh_data])))

In [ ]:
llh_from_pars(multi_hh_data, test_times, 0.015, 2.75)

In [ ]:
tau_vals = arange(0.010, 0.04, 0.005)
lam_vals = arange(2., 4.0, 0.25)
llh_vals = np.zeros((len(tau_vals), len(lam_vals)))
for i in range(len(tau_vals)):
    for j in range(len(lam_vals)):
        llh_vals[i,j] = llh_from_pars(multi_hh_data, test_times, tau_vals[i], lam_vals[j])
        print("tau=",tau_vals[i],", lam=",lam_vals[j], ", llh[tau, lam]=", llh_vals[i, j])

In [ ]:
fig, ax = subplots(1, 1)
lam_inc = lam_vals[1] - lam_vals[0]
lam_max = lam_vals[-1] + (lam_vals[1]-lam_vals[0])
lam_range = lam_max - lam_vals[0]
tau_inc = tau_vals[1] - tau_vals[0]
tau_max = tau_vals[-1] + (tau_vals[1]-tau_vals[0])
tau_range = tau_max - tau_vals[0]
ax.imshow(llh_vals,
            origin='lower',
            extent=(lam_vals[0]-0.5*lam_inc,lam_max-0.5*lam_inc,tau_vals[0]-0.5*tau_inc,tau_max-0.5*tau_inc),
         aspect=lam_range/tau_range)
ax.plot([true_lam],
        [model_input.beta_int],
        marker=".",
        ms=20)

In [ ]:
tau_vals = arange(0.010, 0.04, 0.005)
lam_vals = arange(2., 4.0, 0.25)

llh_fixed_lam = np.zeros((len(tau_vals),))
for i in range(len(tau_vals)):
    print("tau=",tau_vals[i])
    llh_fixed_lam[i] = llh_from_pars(multi_hh_data, test_times, tau_vals[i], true_lam)

llh_fixed_tau = np.zeros((len(lam_vals),))
for i in range(len(lam_vals)):
    print("lam=",lam_vals[i])
    llh_fixed_tau[i] = llh_from_pars(multi_hh_data, test_times, model_input.beta_int, lam_vals[i])

fig, (ax1, ax2) = subplots(1, 2)
ax1.plot(tau_vals, llh_fixed_lam)
ax1.set_xlabel("tau")
ax1.set_ylabel("log likelihood")
ax2.plot(lam_vals, llh_fixed_tau)
ax2.set_xlabel("lambda")
ax2.set_ylabel("log likelihood")

Try using a root finder approach:

In [ ]:
def get_tau_lam_mles(data,test_times,tau_0,lam_0):
    def f(params):
        tau = params[0]
        lam = params[1]
        return -llh_from_pars(data, test_times, tau, lam)

    mle=sp.optimize.minimize(f,[tau_0,lam_0],bounds=((0.005, 0.05),(2., 5.)))
    return mle.x[0],mle.x[1]

In [ ]:
tau_hat, lam_hat = get_tau_lam_mles(multi_hh_data, test_times, 0.02, 2.5)
print("MLE of tau=", tau_hat)
print("MLE of lam=", lam_hat)

New idea: do fits for lots of small households, and see what the distribution of MLE's looks like:

In [ ]:
no_samples = 10
n_hh = 5
tau_mle_list = []
lam_mle_list = []
for smpl_no in range(no_samples):
    print("i=", smpl_no)
    multi_hh_data = [generate_single_hh_test_data(test_times) for i in range(n_hh)]
    tau_hat, lam_hat = get_tau_lam_mles(multi_hh_data, test_times, 0.02, 2.5)
    tau_mle_list.append(tau_hat)
    lam_mle_list.append(lam_hat)